In [1]:
from pydantic import BaseModel, ConfigDict


class User(BaseModel):
    id: int
    name: str = 'Jane Doe'

    model_config = ConfigDict(str_max_length=10)

In [2]:
user = User(id=1, name='John Smith')
print(user)

id=1 name='John Smith'


Using Pydantic, object creation with models requires named argument passing.

In [ ]:
# user = User(1, John Smith')

Using ```model_dump()``` to serialize, or turn into a dict.

In [4]:
assert user.model_dump() == {'id': 1, 'name': 'John Smith'}

### Data Conversion

We may lose some information during pydantic data conversion, like the example below, we went from an input of 3.000 in float, to int of 3.

To fix this, pydantic provides a strict mode, which has several ways to enforcing type matching.

In [4]:
from pydantic import BaseModel


class Model(BaseModel):
    a: int
    b: float
    c: str


print(Model(a=3.000, b='2.72', c=b'binary data').model_dump())
#> {'a': 3, 'b': 2.72, 'c': 'binary data'}

{'a': 3, 'b': 2.72, 'c': 'binary data'}


In [8]:
print(Model(a=3.000, b='2.72', c=b'binary data').model_dump_json(indent=2))

{
  "a": 3,
  "b": 2.72,
  "c": "binary data"
}


Extra fields

By default, Pydantic will ignore extra fields that are not defined in the model. You can change this behavior by setting `extra` in the model's config.

In [10]:
class Model(BaseModel):
    x: int


m = Model(x=1, y='a')
assert m.model_dump() == {'x': 1}

In [11]:
print(m)

x=1


Nested models can be used as well

In [ ]:
from pydantic import BaseModel


class Foo(BaseModel):
    count: int
    size: float | None = None


class Bar(BaseModel):
    apple: str = 'x'
    banana: str = 'y'


class Spam(BaseModel):
    foo: Foo
    bars: list[Bar]


m = Spam(foo={'count': 4}, bars=[{'apple': 'x1'}, {'apple': 'x2'}])
print(m)
#>foo=Foo(count=4, size=None) bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')]
print(m.model_dump())
"""
{
    'foo': {'count': 4, 'size': None},
    'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}],
}
"""

foo=Foo(count=4, size=None) bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')]
{'foo': {'count': 4, 'size': None}, 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}


"\n{\n    'foo': {'count': 4, 'size': None},\n    'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}],\n}\n"

#### Rebuilding model schema

I don't fully get this section is talking about yet.

#### Nested attributes

I don't understand this one yet either.

#### Error handling

Pydantic will raise ```ValidationError``` when data validation fails.

In [1]:
from pydantic import BaseModel, ValidationError


class Model(BaseModel):
    list_of_ints: list[int]
    a_float: float


data = dict(
    list_of_ints=['1', 2, 'bad'],
    a_float='not a float',
)

try:
    Model(**data)
except ValidationError as e:
    print(e)
    """
    2 validation errors for Model
    list_of_ints.2
      Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bad', input_type=str]
    a_float
      Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='not a float', input_type=str]
    """

2 validation errors for Model
list_of_ints.2
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bad', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing
a_float
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='not a float', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_parsing


#### Validating data
Pydantic has three ways to validate data on model classes:
* ```model_validate()``` - very similar to the ```__init__``` method, except this takes a dict or an object
* ```model_validate_json()``` - validate a JSON string or ```bytes``` object. Not sure what ```bytes``` object means.
* ```model_validate_strings()``` - takes a dict that has string keys and string values, and can have nested dict.

In [12]:
from datetime import datetime

from pydantic import BaseModel, ValidationError


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None = None


m = User.model_validate({'id': 123, 'name': 'James'})
print(m)
#> id=123 name='James' signup_ts=None

try:
    User.model_validate(['not', 'a', 'dict'])
except ValidationError as e:
    print(e)
    """
    1 validation error for User
      Input should be a valid dictionary or instance of User [type=model_type, input_value=['not', 'a', 'dict'], input_type=list]
    """

m = User.model_validate_json('{"id": 123, "name": "James"}')
print(m)
#> id=123 name='James' signup_ts=None

try:
    m = User.model_validate_json('{"id": 123, "name": 123}')
except ValidationError as e:
    print(e)
    """
    1 validation error for User
    name
      Input should be a valid string [type=string_type, input_value=123, input_type=int]
    """

try:
    m = User.model_validate_json('invalid JSON')
except ValidationError as e:
    print(e)
    """
    1 validation error for User
      Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='invalid JSON', input_type=str]
    """

m = User.model_validate_strings({'id': '123', 'name': 'James'})
print(m)
#> id=123 name='James' signup_ts=None

m = User.model_validate_strings(
    {'id': '123', 'name': 'James', 'signup_ts': '2024-04-01T12:00:00'}
)
print(m)
#> id=123 name='James' signup_ts=datetime.datetime(2024, 4, 1, 12, 0)

try:
    m = User.model_validate_strings(
        {'id': '123', 'name': 'James', 'signup_ts': '2024-04-01'}, strict=True
    )
except ValidationError as e:
    print(e)
    """
    1 validation error for User
    signup_ts
      Input should be a valid datetime, invalid datetime separator, expected `T`, `t`, `_` or space [type=datetime_parsing, input_value='2024-04-01', input_type=str]
    """

id=123 name='James' signup_ts=None
1 validation error for User
  Input should be a valid dictionary or instance of User [type=model_type, input_value=['not', 'a', 'dict'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
id=123 name='James' signup_ts=None
1 validation error for User
name
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
1 validation error for User
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='invalid JSON', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid
id=123 name='James' signup_ts=None
id=123 name='James' signup_ts=datetime.datetime(2024, 4, 1, 12, 0)
1 validation error for User
signup_ts
  Input should be a valid datetime, invalid datetime separator, expected `T`, `t`, `_` or space [type=datetime_parsing, input_value='

#### Creating models without validation
There are ways to create pydatnic models without validation, possibly in cases like these:
* complex data that is already known to be valid, we can skip validation for perf reasons
* when validator function are non-idempotent
* when validator function have side effects (doesn't this mean the same as last point?)